In [1]:
import os
import time
import numpy as np
import tensorflow as tf
from tensorflow.keras import layers, models, regularizers, Model
from tensorflow.keras.applications import (
    MobileNetV3Small, ResNet50, EfficientNetB0, DenseNet121
)
from tensorflow.keras.applications import MobileNet
# from tensorflow.keras.applications.vit import ViT
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.utils import image_dataset_from_directory
from sklearn.metrics import (
    classification_report, confusion_matrix, cohen_kappa_score,
    matthews_corrcoef, f1_score, precision_score, recall_score,
    roc_curve, auc
)
import matplotlib.pyplot as plt
import seaborn as sns
import pandas as pd
from sklearn.preprocessing import label_binarize
from tensorflow.python.framework.convert_to_constants import convert_variables_to_constants_v2_as_graph

# === 1. Data Loading ===
data_folder = "/kaggle/input/pancrease-ct-segmenatation/images"
img_size = (224, 224)
batch_size = 16
AUTOTUNE = tf.data.AUTOTUNE

ds = image_dataset_from_directory(data_folder, image_size=img_size, batch_size=batch_size, label_mode='int')
class_names = ds.class_names
num_classes = len(class_names)

ds = ds.shuffle(1000).cache().prefetch(buffer_size=AUTOTUNE)
val_size = int(0.2 * tf.data.experimental.cardinality(ds).numpy())
val_ds = ds.take(val_size)
train_ds = ds.skip(val_size)

# === 2. Custom Hybrid Model ===
class CBAM(layers.Layer):
    def __init__(self, filters, reduction=16, **kwargs):
        super(CBAM, self).__init__(**kwargs)
        self.avg_pool = layers.GlobalAveragePooling2D()
        self.max_pool = layers.GlobalMaxPooling2D()
        self.shared_dense_one = layers.Dense(filters // reduction, activation='relu', use_bias=False)
        self.shared_dense_two = layers.Dense(filters, use_bias=False)
        self.sigmoid = layers.Activation('sigmoid')

    def call(self, inputs):
        avg_out = self.shared_dense_two(self.shared_dense_one(self.avg_pool(inputs)))
        max_out = self.shared_dense_two(self.shared_dense_one(self.max_pool(inputs)))
        scale = self.sigmoid(avg_out + max_out)
        scale = tf.expand_dims(tf.expand_dims(scale, axis=1), axis=1)
        return inputs * scale

class LASERAttention(layers.Layer):
    def __init__(self, embed_dim, num_heads, rank=16):
        super(LASERAttention, self).__init__()
        assert embed_dim % num_heads == 0, "embed_dim must be divisible by num_heads"
        self.embed_dim = embed_dim
        self.num_heads = num_heads
        self.rank = rank
        self.proj_dim = embed_dim // num_heads
        self.q_proj = layers.Dense(embed_dim)
        self.k_proj = layers.Dense(embed_dim)
        self.v_proj = layers.Dense(embed_dim)
        self.out_proj = layers.Dense(embed_dim)
        self.U = self.add_weight(shape=(num_heads, rank, self.proj_dim), initializer="glorot_uniform", trainable=True)
        self.V = self.add_weight(shape=(num_heads, rank, self.proj_dim), initializer="glorot_uniform", trainable=True)

    def call(self, inputs):
        batch_size = tf.shape(inputs)[0]
        seq_len = tf.shape(inputs)[1]

        Q = tf.reshape(self.q_proj(inputs), (batch_size, seq_len, self.num_heads, self.proj_dim))
        K = tf.reshape(self.k_proj(inputs), (batch_size, seq_len, self.num_heads, self.proj_dim))
        V = tf.reshape(self.v_proj(inputs), (batch_size, seq_len, self.num_heads, self.proj_dim))

        Q = tf.transpose(Q, perm=[0, 2, 1, 3])
        K = tf.transpose(K, perm=[0, 2, 1, 3])
        V = tf.transpose(V, perm=[0, 2, 1, 3])

        Q_lowrank = tf.einsum('bhqd,hrd->bhrq', Q, self.U)
        Q_lowrank = tf.transpose(Q_lowrank, perm=[0, 1, 3, 2])
        K_lowrank = tf.einsum('bhkd,hrd->bhrk', K, self.V)

        attn = tf.matmul(Q_lowrank, K_lowrank) / tf.math.sqrt(tf.cast(self.rank, tf.float32))
        attn = tf.nn.softmax(attn, axis=-1)
        out = tf.matmul(attn, V)

        out = tf.transpose(out, perm=[0, 2, 1, 3])
        out = tf.reshape(out, (batch_size, seq_len, self.embed_dim))
        
        return self.out_proj(out)

class TransformerEncoderLASERAttention(layers.Layer):
    def __init__(self, embed_dim, num_heads, ff_dim, rank=16):
        super(TransformerEncoderLASERAttention, self).__init__()
        self.attention = LASERAttention(embed_dim, num_heads, rank=rank)
        self.ffn = tf.keras.Sequential([
            layers.Dense(ff_dim, activation="relu"),
            layers.Dense(embed_dim),
        ])
        self.layernorm1 = layers.LayerNormalization(epsilon=1e-6)
        self.layernorm2 = layers.LayerNormalization(epsilon=1e-6)
        self.add1 = layers.Add()
        self.add2 = layers.Add()

    def call(self, inputs):
        attn_output = self.attention(inputs)
        out1 = self.layernorm1(self.add1([inputs, attn_output]))
        ffn_output = self.ffn(out1)
        return self.layernorm2(self.add2([out1, ffn_output]))

class CTIModule(layers.Layer):
    def __init__(self, output_dim):
        super(CTIModule, self).__init__()
        self.dense = layers.Dense(output_dim, activation='relu')

    def call(self, inputs):
        return self.dense(inputs)

def HybridModel(input_shape=(224,224,3), num_classes=2, embed_dim=128, num_heads=8, ff_dim=256):
    inputs = layers.Input(shape=input_shape)
    mobilenet = MobileNetV3Small(include_top=False, input_shape=input_shape, weights='imagenet')
    x = mobilenet(inputs)
    x = CBAM(x.shape[-1])(x)
    x = layers.Conv2D(embed_dim, kernel_size=1)(x)

    def reshape_for_transformer(t):
        shape = tf.shape(t)
        return tf.reshape(t, (shape[0], shape[1]*shape[2], shape[3]))

    x_reshape = layers.Lambda(reshape_for_transformer)(x)
    x_transformed = TransformerEncoderLASERAttention(embed_dim, num_heads, ff_dim)(x_reshape)
    x_pooled = layers.GlobalAveragePooling1D()(x_transformed)
    cnn_pooled = layers.GlobalAveragePooling2D()(x)

    fused = layers.Concatenate()([cnn_pooled, x_pooled])
    fused = CTIModule(embed_dim)(fused)
    x = layers.Dropout(0.5)(fused)
    x = layers.Dense(256, activation='relu', kernel_regularizer=regularizers.l2(0.001))(x)
    x = layers.Dropout(0.3)(x)
    outputs = layers.Dense(num_classes, activation='sigmoid')(x)
    return Model(inputs, outputs)

def build_model(base_model, num_classes):
    inputs = tf.keras.Input(shape=(224, 224, 3))
    x = base_model(inputs, training=False)
    x = layers.GlobalAveragePooling2D()(x)
    x = layers.Dense(128, activation='relu')(x)
    x = layers.Dense(num_classes, activation='sigmoid')(x)
    return tf.keras.Model(inputs, x)




# Build and summarize the model
model = HybridModel(num_classes=num_classes)
model.summary()


2025-09-07 12:13:48.259554: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1757247228.608343      36 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1757247228.707860      36 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered


Found 18942 files belonging to 2 classes.


I0000 00:00:1757247284.155037      36 gpu_device.cc:2022] Created device /job:localhost/replica:0/task:0/device:GPU:0 with 13942 MB memory:  -> device: 0, name: Tesla T4, pci bus id: 0000:00:04.0, compute capability: 7.5
I0000 00:00:1757247284.155808      36 gpu_device.cc:2022] Created device /job:localhost/replica:0/task:0/device:GPU:1 with 13942 MB memory:  -> device: 1, name: Tesla T4, pci bus id: 0000:00:05.0, compute capability: 7.5


4334752/4334752 ━━━━━━━━━━━━━━━━━━━━ 0s 0us/step


Model: "functional_1"

┏━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┓
┃ Layer (type)        ┃ Output Shape      ┃    Param # ┃ Connected to      ┃
┡━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━┩
│ input_layer         │ (None, 224, 224,  │          0 │ -                 │
│ (InputLayer)        │ 3)                │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ MobileNetV3Small    │ (None, 7, 7, 576) │    939,120 │ input_layer[0][0] │
│ (Functional)        │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ cbam (CBAM)         │ (None, 7, 7, 576) │     41,472 │ MobileNetV3Small… │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2d (Conv2D)     │ (None, 7, 7, 128) │     73,856 │ cbam[0][0]        │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ lambda (Lambda)     │ (None, 49, 128)   │          0 │ conv2d[0][0]      │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ transformer_encode… │ (None, 49, 128)   │    136,576 │ lambda[0][0]      │
│ (TransformerEncode… │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ global_average_poo… │ (None, 128)       │          0 │ conv2d[0][0]      │
│ (GlobalAveragePool… │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ global_average_poo… │ (None, 128)       │          0 │ transformer_enco… │
│ (GlobalAveragePool… │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ concatenate         │ (None, 256)       │          0 │ global_average_p… │
│ (Concatenate)       │                   │            │ global_average_p… │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ cti_module          │ (None, 128)       │     32,896 │ concatenate[0][0] │
│ (CTIModule)         │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ dropout (Dropout)   │ (None, 128)       │          0 │ cti_module[0][0]  │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ dense_9 (Dense)     │ (None, 256)       │     33,024 │ dropout[0][0]     │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ dropout_1 (Dropout) │ (None, 256)       │          0 │ dense_9[0][0]     │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ dense_10 (Dense)    │ (None, 2)         │        514 │ dropout_1[0][0]   │
└─────────────────────┴───────────────────┴────────────┴───────────────────┘

 Total params: 1,257,458 (4.80 MB)

 Trainable params: 1,245,346 (4.75 MB)

 Non-trainable params: 12,112 (47.31 KB)